![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FDev&dt=CPR+Endpoints+That+Train.ipynb)

# Custom Prediction Routines for Endpoints That Train

- https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines
- https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.prediction.LocalModel
- https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/predictor.py
- https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/sklearn/predictor.py
- https://codelabs.developers.google.com/vertex-cpr-sklearn#0
- prediction request limit is 1.5MB: https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#send_an_online_prediction_request

**TODO**
- [ ] Try using storage read for BigQuery
    - https://cloud.google.com/bigquery/docs/reference/storage/libraries#client-libraries-usage-python

In [29]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [30]:
REGION = 'us-central1'
EXPERIMENT = 'cpr_training'
SERIES = '04'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

In [31]:
from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import service_usage_v1

from datetime import datetime
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn import metrics

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

In [32]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project = PROJECT_ID)

In [33]:
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PROJECT_ID}"
DIR = f"temp/{EXPERIMENT}"

In [34]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

In [35]:
!rm -rf {DIR}
!mkdir -p {DIR}

## Idea: Decision Tree on Samples

- Input parameter is a sample size `n`
- Retrieve a sample of size `n` from a BigQuery table to a Pandas dataframe
- Use sklearn.tree.DecisionTreeClassifier to build a classifier
- Retrieve the rules of the tree

In [36]:
n = 4000

In [37]:
train = bq.query(query = f"""
        SELECT * EXCEPT(splits, transaction_id)
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
            WHERE splits = 'TRAIN' and RAND() < 0.1
            LIMIT {n}
        """).to_dataframe()
y = train[VAR_TARGET]
X = train.drop(VAR_TARGET, axis = 1)

In [38]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_leaf_nodes = 3, random_state = 0)
clf.fit(X, y)

DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)

In [39]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

def path_builder(node_num, path, x):
        path.append(node_num)
        if node_num == x:
            return True
        left = False
        right = False
        if (children_left[node_num] !=-1):
            left = path_builder(children_left[node_num], path, x)
        if (children_right[node_num] !=-1):
            right = path_builder(children_right[node_num], path, x)
        if left or right :
            return True
        path.remove(node_num)
        return False


def rule_builder(path, column_names):
    rule = ''
    for index, node in enumerate(path):
        if index != len(path)-1:
            if len(rule) > 0: rule += ' and '
            if (children_left[node] == path[index+1]):
                rule += f"{column_names[feature[node]]} <= {threshold[node]}"
            else:
                rule += f"{column_names[feature[node]]} > {threshold[node]}"
    return rule

paths ={}
for leaf in np.unique(clf.apply(X)):
    path_leaf = []
    path_builder(0, path_leaf, leaf)
    paths[leaf] = np.unique(np.sort(path_leaf))

rules = {}
for key in paths:
    rules[key] = rule_builder(paths[key], X.columns)

rules

{2: 'V11 > 3.3807291984558105',
 3: 'V11 <= 3.3807291984558105 and V16 <= -3.9015995264053345',
 4: 'V11 <= 3.3807291984558105 and V16 > -3.9015995264053345'}

---
## Build Custom Prediction Routine

A custom container built by the Vertex AI SDK that assist with pre/post processing code without the need to setup an HTTP server.

In [13]:
!pip install google-cloud-aiplatform[prediction] -U -q

In [40]:
!mkdir -p {DIR}/SRC

In [15]:
%%writefile {DIR}/SRC/requirements.txt
fastapi
uvicorn==0.17.6
#joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
pandas
#google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0
google-cloud-bigquery
pyarrow

Writing temp/cpr_training/SRC/requirements.txt


In [16]:
%%writefile {DIR}/SRC/predictor.py

# packages
import numpy as np
import json
from sklearn.tree import DecisionTreeClassifier
from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud import bigquery

##################################################################################################

# clients
bq = bigquery.Client(project = 'statmike-mlops-349915')

# source data
BQ_PROJECT = 'statmike-mlops-349915'
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Model Training
VAR_TARGET = 'Class'

def ruler(n):
    # helper function: 
    def path_builder(node_num, path, x):
            path.append(node_num)
            if node_num == x:
                return True
            left = False
            right = False
            if (children_left[node_num] !=-1):
                left = path_builder(children_left[node_num], path, x)
            if (children_right[node_num] !=-1):
                right = path_builder(children_right[node_num], path, x)
            if left or right :
                return True
            path.remove(node_num)
            return False
    # helper function:
    def rule_builder(path, column_names):
        rule = ''
        for index, node in enumerate(path):
            if index != len(path)-1:
                if len(rule) > 0: rule += ' and '
                if (children_left[node] == path[index+1]):
                    rule += f"{column_names[feature[node]]} <= {threshold[node]}"
                else:
                    rule += f"{column_names[feature[node]]} > {threshold[node]}"
        return rule
    
    # data
    train = bq.query(query = f"""
        SELECT * EXCEPT(splits, transaction_id)
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
            WHERE splits = 'TRAIN' and RAND() < 0.1
            LIMIT {n}
        """).to_dataframe()
    y = train[VAR_TARGET]
    X = train.drop(VAR_TARGET, axis = 1)
    
    # model
    clf = DecisionTreeClassifier(max_leaf_nodes = 3, random_state = 0)
    clf.fit(X, y)
    
    # outputs
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold
    
    # decision
    paths ={}
    for leaf in np.unique(clf.apply(X)):
        path_leaf = []
        path_builder(0, path_leaf, leaf)
        paths[leaf] = np.unique(np.sort(path_leaf))
    rules = {}
    for key in paths:
        rules[key] = rule_builder(paths[key], X.columns)

    return rules

##################################################################################################

class CprPredictor(Predictor):
    def __init__(self):
        return

    def load(self, artifacts_uri: str) -> None:
        # no model to load here, this example trains a model and returns parameters
        pass

    def predict(self, instances):

        instances = instances["instances"]
        results = [f"{ruler(instance)}" for instance in instances]
        
        return {"predictions": results}

Writing temp/cpr_training/SRC/predictor.py


In [17]:
from google.cloud.aiplatform.prediction import LocalModel
# load the local predictor class:
from temp.cpr_training.SRC.predictor import CprPredictor

local_model = LocalModel.build_cpr_model(
    src_dir = f"{DIR}/SRC",
    output_image_uri = f"{REPOSITORY}/{SERIES}_{EXPERIMENT}",
    predictor = CprPredictor,
    requirements_path = os.path.join(f"{DIR}/SRC", "requirements.txt"),
)

In [41]:
with local_model.deploy_to_local_endpoint() as local_endpoint:
    predict_response = local_endpoint.predict(
        request = '{"instances": [100, 1000, 2000, 3000]}',
        headers={"Content-Type": "application/json"}
    )

    health_check_response = local_endpoint.run_health_check()

In [42]:
[print(r+'\n') for r in json.loads(predict_response.content)['predictions']]

{1: 'V11 <= 1.930665373802185', 2: 'V11 > 1.930665373802185'}

{1: 'V3 <= -5.268670558929443', 2: 'V3 > -5.268670558929443'}

{1: 'V10 <= -2.929439663887024', 3: 'V10 > -2.929439663887024 and V4 <= 5.794674873352051', 4: 'V10 > -2.929439663887024 and V4 > 5.794674873352051'}

{1: 'V17 <= -3.3473413586616516', 2: 'V17 > -3.3473413586616516'}



[None, None, None, None]

In [20]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [21]:
local_model.push_image()

In [22]:
model = aiplatform.Model.upload(
    local_model = local_model,
    display_name = f"{SERIES}_{EXPERIMENT}"
)

Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/7595921104909107200/operations/3630419513234685952
Model created. Resource name: projects/1026793852137/locations/us-central1/models/7595921104909107200@1
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/7595921104909107200@1')


In [23]:
#model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/6342900061709533184@1')

In [24]:
endpoint = model.deploy(
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 5,
    service_account = SERVICE_ACCOUNT
)

Creating Endpoint
Create Endpoint backing LRO: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768/operations/7667896579172335616
Endpoint created. Resource name: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1026793852137/locations/us-central1/endpoints/5561404580081696768')
Deploying model to Endpoint : projects/1026793852137/locations/us-central1/endpoints/5561404580081696768
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768/operations/685065356934381568
Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768


In [25]:
#endpoint = aiplatform.Endpoint('projects/1026793852137/locations/us-central1/endpoints/4685331307255824384')

In [43]:
predictions = endpoint.predict(instances = [100, 1000, 2000])
predictions

Prediction(predictions=["{1: 'V17 <= -7.469338417053223', 3: 'V17 > -7.469338417053223 and V17 <= 1.427743911743164', 4: 'V17 > -7.469338417053223 and V17 > 1.427743911743164'}", "{1: 'V14 <= -5.702228307723999', 2: 'V14 > -5.702228307723999'}", "{1: 'V10 <= -2.821014881134033', 2: 'V10 > -2.821014881134033'}"], deployed_model_id='4246379877168578560', model_version_id='1', model_resource_name='projects/1026793852137/locations/us-central1/models/7595921104909107200', explanations=None)

In [44]:
len(predictions.predictions)

3

In [28]:
endpoint.resource_name

'projects/1026793852137/locations/us-central1/endpoints/5561404580081696768'

In [45]:
# get the Async Client for the endpoint:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
parent = f"projects/{PROJECT_ID}/locations/{REGION}"

client = aiplatform_v1.PredictionServiceAsyncClient(client_options = client_options)

In [46]:
instance = [1000]

In [47]:
import asyncio
import time

In [48]:
predictions = await client.predict(endpoint = endpoint.resource_name, instances = instance)
predictions

predictions {
  string_value: "{1: \'V17 <= -5.97195303440094\', 2: \'V17 > -5.97195303440094\'}"
}
deployed_model_id: "4246379877168578560"
model: "projects/1026793852137/locations/us-central1/models/7595921104909107200"
model_display_name: "04_cpr_training"
model_version_id: "1"

In [49]:
len(predictions.predictions)

1

In [50]:
predictions.predictions

["{1: 'V17 <= -5.97195303440094', 2: 'V17 > -5.97195303440094'}"]

In [55]:
async def asyncPredictions(batch_size = 1, concur_requests = 10, total_instances = 100):
    limit = asyncio.Semaphore(concur_requests)
    instance = [1000]
    predictions = [None] * total_instances

    async def predictor(p, batch_size):
        async with limit:
            if limit.locked():
                await asyncio.sleep(.01)
            prediction = await client.predict(
                endpoint = endpoint.resource_name, 
                instances = instance * batch_size,
                #retry = my_retry_policy,
                timeout = 100000
            )

        predictions[p:p+batch_size] = prediction.predictions

    async def runner(batch_size, total_instances):
        tasks = []
        for p in range(0, total_instances, batch_size):
            task = asyncio.create_task(predictor(p, batch_size))
            tasks.append(task)
        results = await asyncio.gather(*tasks)

    start = time.perf_counter()
    await runner(batch_size, total_instances)
    elapsed = time.perf_counter() - start
    print(f'{elapsed:0.5f} seconds')
    
    return predictions

In [56]:
predictions = await asyncPredictions(1, 2, 10)

9.92153 seconds


In [57]:
len(predictions)

10

In [58]:
predictions

["{1: 'V11 <= 3.8927831649780273', 2: 'V11 > 3.8927831649780273'}",
 "{1: 'V3 <= -4.695509910583496', 2: 'V3 > -4.695509910583496'}",
 "{1: 'V11 <= 3.341222047805786', 2: 'V11 > 3.341222047805786'}",
 "{1: 'V3 <= -4.998672723770142', 3: 'V3 > -4.998672723770142 and V4 <= 6.29348349571228', 4: 'V3 > -4.998672723770142 and V4 > 6.29348349571228'}",
 "{1: 'V17 <= -6.767475247383118', 3: 'V17 > -6.767475247383118 and V11 <= 2.122326135635376', 4: 'V17 > -6.767475247383118 and V11 > 2.122326135635376'}",
 "{1: 'V11 <= 3.2266104221343994', 2: 'V11 > 3.2266104221343994'}",
 "{1: 'V3 <= -5.491417169570923', 2: 'V3 > -5.491417169570923'}",
 "{1: 'V14 <= -4.806050777435303', 2: 'V14 > -4.806050777435303'}",
 "{0: ''}",
 "{0: ''}"]

In [59]:
predictions = await asyncPredictions(1, 4, 20)

9.67121 seconds


In [60]:
len(predictions)

20

In [489]:
predictions = await asyncPredictions(10, 4, 40)

29.90747 seconds


In [490]:
len(predictions)

40

In [525]:
predictions = await asyncPredictions(10, 4, 100)

65.90143 seconds


In [526]:
len(predictions)

100

In [527]:
predictions = await asyncPredictions(10, 5, 100)

62.64738 seconds


In [528]:
len(predictions)

100

In [495]:
predictions = await asyncPredictions(10, 6, 100)

50.71739 seconds


In [496]:
len(predictions)

100

In [497]:
predictions = await asyncPredictions(10, 7, 100)

60.53520 seconds


In [498]:
len(predictions)

100

In [506]:
predictions = await asyncPredictions(10, 8, 100)

60.52704 seconds


In [507]:
len(predictions)

100

In [120]:
from google.api_core.retry import Retry
from google.api_core import exceptions

_RETRIABLE_TYPES = [
    exceptions.InternalServerError,
    exceptions.BadGateway,
    exceptions.TooManyRequests,
    exceptions.ServiceUnavailable,
]

def is_retryable(exc):
    return isinstance(exc, _RETRIABLE_TYPES)

my_retry_policy = Retry(predicate = is_retryable)

In [136]:
async def predictor(batch_size, n):
    prediction = await client.predict(
        endpoint = endpoint.resource_name, 
        instances = [n] * batch_size,
        retry = Retry(deadline = 120),
        timeout = 100000)
    return prediction.predictions[0]

async def runner(concur_requests, batch_size, n):
    tasks = [predictor(batch_size, n) for _ in range(concur_requests)]
    return await asyncio.gather(*tasks)

In [130]:
results = await runner(10, 1, 1000)
len(results)

10

In [131]:
results

["{1: 'V3 <= -5.327307939529419', 3: 'V3 > -5.327307939529419 and V11 <= 2.131626844406128', 4: 'V3 > -5.327307939529419 and V11 > 2.131626844406128'}",
 "{0: ''}",
 "{1: 'V17 <= -2.4286970496177673', 3: 'V17 > -2.4286970496177673 and V13 <= -2.413966417312622', 4: 'V17 > -2.4286970496177673 and V13 > -2.413966417312622'}",
 "{1: 'V10 <= -2.696150541305542', 2: 'V10 > -2.696150541305542'}",
 "{1: 'V11 <= 3.3314086198806763', 2: 'V11 > 3.3314086198806763'}",
 "{1: 'V3 <= -4.791704416275024', 3: 'V3 > -4.791704416275024 and V4 <= 6.180951833724976', 4: 'V3 > -4.791704416275024 and V4 > 6.180951833724976'}",
 "{1: 'V17 <= -5.151315927505493', 3: 'V17 > -5.151315927505493 and V13 <= -2.407018303871155', 4: 'V17 > -5.151315927505493 and V13 > -2.407018303871155'}",
 "{0: ''}",
 "{1: 'V10 <= -2.6969146728515625', 2: 'V10 > -2.6969146728515625'}",
 "{2: 'V11 > 3.078174352645874', 3: 'V11 <= 3.078174352645874 and V7 <= 1.2924550771713257', 4: 'V11 <= 3.078174352645874 and V7 > 1.29245507717132

In [137]:
results = []
for l in range(2):
    results += await runner(10, 1, 1000)
len(results)

20

In [138]:
results

["{1: 'V10 <= -2.87595272064209', 2: 'V10 > -2.87595272064209'}",
 "{1: 'V4 <= 4.793796539306641', 3: 'V4 > 4.793796539306641 and V16 <= 1.2312788218259811', 4: 'V4 > 4.793796539306641 and V16 > 1.2312788218259811'}",
 "{1: 'V11 <= 3.702981114387512', 2: 'V11 > 3.702981114387512'}",
 "{1: 'V11 <= 2.8443214893341064', 2: 'V11 > 2.8443214893341064'}",
 "{1: 'V10 <= -2.585103750228882', 3: 'V10 > -2.585103750228882 and V7 <= 1.2888912558555603', 4: 'V10 > -2.585103750228882 and V7 > 1.2888912558555603'}",
 "{1: 'V17 <= -3.8000799417495728', 3: 'V17 > -3.8000799417495728 and V7 <= 1.2912284135818481', 4: 'V17 > -3.8000799417495728 and V7 > 1.2912284135818481'}",
 "{1: 'V14 <= -4.832205533981323', 3: 'V14 > -4.832205533981323 and V13 <= -2.4244229793548584', 4: 'V14 > -4.832205533981323 and V13 > -2.4244229793548584'}",
 "{0: ''}",
 "{1: 'V6 <= -3.1283116340637207', 2: 'V6 > -3.1283116340637207'}",
 "{1: 'V7 <= 1.2913868427276611', 3: 'V7 > 1.2913868427276611 and V7 <= 1.2960330247879028', 

In [139]:
results = []
for l in range(1):
    results += await runner(25, 1, 4000)
len(results)

25

In [142]:
results = []
for l in range(1):
    results += await runner(25, 1, 40000)
len(results)

ServiceUnavailable: 503 502:Bad Gateway

In [61]:
endpoint.delete(force = True)

Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768/operations/626905589871542272
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768
Deleting Endpoint : projects/1026793852137/locations/us-central1/endpoints/5561404580081696768
Delete Endpoint  backing LRO: projects/1026793852137/locations/us-central1/operations/2364169146129711104
Endpoint deleted. . Resource name: projects/1026793852137/locations/us-central1/endpoints/5561404580081696768


In [62]:
model.delete()

Deleting Model : projects/1026793852137/locations/us-central1/models/7595921104909107200
Delete Model  backing LRO: projects/1026793852137/locations/us-central1/operations/999578459036450816
Model deleted. . Resource name: projects/1026793852137/locations/us-central1/models/7595921104909107200
